In [2]:
import pickle
import pandas as pd
import torch
import io

from src.quantization.quantized_model import QuantizedModel
from src.exploration.problems import LayerwiseQuantizationProblem

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)


with open('../results/exploration_resnet18_imagenet_2022-11-08_13-46-00.pkl', 'rb') as f:
    d = CPU_Unpickler(f).load()

type(d)


/home/oq4116/virtualenvs/torch_quantization/lib64/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pymoo.core.result.Result

In [3]:
print(d.problem.min_accuracy)

0.7


In [13]:
import numpy as np

accuracy_bound = d.problem.min_accuracy
shape_x = d.problem.n_obj + d.problem.n_constr + d.problem.n_var

all_individuals = np.ndarray( (0, shape_x) )

for h in d.history:
    for ind in h.opt:
        individual_row = np.concatenate( (ind.get("G") - accuracy_bound, ind.get("F"), ind.get("X")) )
        individual_row = np.expand_dims(individual_row, axis=0)
        all_individuals = np.concatenate( (all_individuals, individual_row), axis=0)
        
df = pd.DataFrame(all_individuals)
df[0] = -df[0] # invert Accuracy
df_fit = df.where(df[0] > accuracy_bound) 



In [14]:
print(df_fit)

           0         1      2    3     4     5     6     7     8     9   ...  \
0         NaN       NaN    NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1    0.702881 -0.702881  473.0  9.0  15.0  13.0  14.0  13.0  12.0  14.0  ...   
2    0.702881 -0.702881  473.0  9.0  15.0  13.0  14.0  13.0  12.0  14.0  ...   
3    0.703003 -0.703003  489.0  6.0  12.0  13.0  14.0  13.0  12.0  14.0  ...   
4    0.733032 -0.733032  491.0  9.0  15.0  13.0  14.0  13.0  12.0  14.0  ...   
..        ...       ...    ...  ...   ...   ...   ...   ...   ...   ...  ...   
132  0.734009 -0.734009  415.0  9.0  11.0  10.0   8.0  12.0   8.0  14.0  ...   
133  0.738159 -0.738159  446.0  9.0  11.0  10.0   8.0  12.0  12.0  14.0  ...   
134  0.727539 -0.727539  409.0  9.0  12.0  10.0   8.0  12.0   8.0  14.0  ...   
135  0.735107 -0.735107  439.0  9.0  11.0  10.0   8.0  12.0  12.0  14.0  ...   
136  0.727051 -0.727051  404.0  9.0  11.0   8.0   8.0  11.0   8.0  14.0  ...   

       36    37    38    39    40    41

In [7]:
df.sort_values(1)

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
133,0.738159,-0.738159,446.0,9.0,11.0,10.0,8.0,12.0,12.0,14.0,...,13.0,10.0,12.0,7.0,10.0,7.0,6.0,15.0,7.0,13.0
128,0.738159,-0.738159,446.0,9.0,11.0,10.0,8.0,12.0,12.0,14.0,...,13.0,10.0,12.0,7.0,10.0,7.0,6.0,15.0,7.0,13.0
94,0.737671,-0.737671,485.0,9.0,15.0,13.0,14.0,13.0,12.0,14.0,...,13.0,10.0,12.0,7.0,10.0,9.0,16.0,16.0,7.0,13.0
101,0.737671,-0.737671,485.0,9.0,15.0,13.0,14.0,13.0,12.0,14.0,...,13.0,10.0,12.0,7.0,10.0,9.0,16.0,16.0,7.0,13.0
109,0.737671,-0.737671,485.0,9.0,15.0,13.0,14.0,13.0,12.0,14.0,...,13.0,10.0,12.0,7.0,10.0,9.0,16.0,16.0,7.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.702026,-0.702026,411.0,9.0,12.0,10.0,8.0,11.0,8.0,14.0,...,13.0,10.0,7.0,6.0,10.0,7.0,3.0,15.0,7.0,13.0
55,0.701294,-0.701294,411.0,9.0,12.0,10.0,8.0,11.0,8.0,14.0,...,13.0,10.0,7.0,6.0,10.0,7.0,3.0,15.0,7.0,13.0
62,0.701294,-0.701294,411.0,9.0,12.0,10.0,8.0,11.0,8.0,14.0,...,13.0,10.0,7.0,6.0,10.0,7.0,3.0,15.0,7.0,13.0
103,0.700684,-0.700684,408.0,9.0,12.0,10.0,8.0,10.0,8.0,14.0,...,13.0,10.0,7.0,6.0,10.0,7.0,3.0,15.0,7.0,13.0


In [1]:
from torchvision import transforms
import webdataset as wds

def identity(d):
    return d

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

transf = transforms.Compose([transforms.Resize(256),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            normalize])

train_path = '/tools/datasets/imagenet/train/imagenet-train-{0000..0136}.tar'

dataset_load = (
    wds.WebDataset(train_path, shardshuffle=True)
    .decode("pil")
    .to_tuple("input.jpeg", "target.cls")
    .shuffle(10000)
    .map_tuple(transf, identity)
)

In [6]:
next(iter(dataset_load))[1]

529

In [10]:
from torch.utils.data import DataLoader

print(type(dataset_load))
dataset_load = dataset_load.batched(16)

loader = wds.WebLoader(dataset_load, num_workers=2, batch_size=None)
loader = loader.unbatched().shuffle(1000).batched(16)
# loader = DataLoader(dataset_load, num_workers=2, batch_size=16)

batch = next(iter(loader))
print(batch[1])



<class 'webdataset.compat.WebDataset'>
tensor([[[ 78, 121, 265,  ..., 914, 914, 265],
         [121, 121, 774,  ..., 121, 265, 774],
         [121, 265, 121,  ..., 121, 121, 121],
         ...,
         [265, 774, 265,  ..., 914,  78, 265],
         [914, 914, 152,  ..., 265, 265, 265],
         [121, 774, 914,  ..., 914, 121, 774]],

        [[143, 680, 674,  ..., 674, 927, 830],
         [ 41, 618, 456,  ...,  69, 674,  69],
         [401, 703, 383,  ..., 618, 830, 638],
         ...,
         [927, 638, 680,  ..., 674, 830, 835],
         [680,  41, 680,  ..., 674, 680, 703],
         [674, 456, 349,  ..., 383, 830, 680]],

        [[ 84, 461, 602,  ..., 934, 602, 265],
         [509, 823, 265,  ..., 461, 573, 602],
         [774, 854, 956,  ..., 823, 461, 509],
         ...,
         [914, 774, 509,  ..., 934, 461, 823],
         [509, 461, 823,  ..., 368, 121, 573],
         [854,  78, 461,  ..., 398, 774, 368]],

        ...,

        [[956, 854, 602,  ..., 774, 886, 823],
      